## CS/INFO 5304 Assignment 2: Recommender Systems

#### Credit: 35 points + possible bonus (10 points)
#### Due date: May 2nd, 11:59PM
 
The goal of the assignment is to get familiar with different types of recommender systems. Specifically, we are going to build a system that can recommend Yelp businesses to users.
 
 
### Dataset
The dataset that we will be using contains information about Yelp businesses. More precisely, for 14397 active users and 1000 popular businesses on Yelp, we know if a given user visited&rated a given business, for example, a restaurant.
 
 
The folder contains:
 
- __user-business.csv__ - This is the ratings matrix R, where each row corresponds to a user and each column corresponds to a business. Rij = 1 if the user has visited&rated that business. Otherwise  Rij = 0. (To simplify the question, we ignore the exact ratings.) The columns are separated by a space.
 
- __business.csv__ - This is a file containing the names of the businesses, in the same order as the columns of R.
 
The dataset can be found here: Google drive.
  

### Overview
 
In this assignment we are going to implement three types of recommender systems namely
 
- __User__ - User recommender system
- __Item__ – Item recommender system
- Latent factor model recommender system
 
 
We are then going to compare the results of these systems for the 4th user(index starting from 1) of the dataset. Let’s call him Alex. In order to do so, we have erased the first 100 entries of Alex’s row in the matrix, and replaced them by 0s. This means that we don’t know which of the first 100 businesses Alex has visited. Based on Alex’s behavior on the other businesses, you need to give Alex recommendations on the first 100 businesses. We will then see if our recommendations match what Alex had in fact visited.

To verify your output using the following recommenders, the 1s in the erased first entries are:
- Piece of Cake
- Papi's Cuban & Caribbean Grill
- Loca Luna
- Farm Burger
- Little Rey
- Seven Lamps
- Vatica Indian Cuisine
- Shake Shack
- Truva Turkish Kitchen
- Yoi Yoi Japanese Steakhouse & Sushi

In [1]:
def display_top_k(business_df, recom_mat, k = 5):
    df = pd.DataFrame(index = range(1, 6))

    top_k_idxs = recom_mat.argsort()[-k:][::-1]

    df["Business"] = np.array(business_df.iloc[top_k_idxs, 0])
    df["Similarity Score"] = np.array(recom_mat[top_k_idxs])

    display(df)

### Part A: user – user recommender system [10 points]
 
In a user-user recommender system, you need to find users who have visited and rated similar businesses. Then among these users, you can recommend the top visited items.

for all businesses b, compute

$$ r_{Alex,b} = \sum_{x∈users} cos-sim(x, Alex)·R_{x,b} $$

where $cos-sim(x,Alex)$ is the cosine similarity of other users with Alex (excluding entries of the first 100 businesses), and $R$ is the ratings matrix. 

In the above equation you are first finding the similarity between users and then multiplying it with their product rating for each item. So the businesses that have higher $r_{Alex,b}$ will be the businesses that are popular among the users similar to Alex.
 
Let $S$ denote the set of the first 100 businesses (the first 100 columns of the matrix). From all the businesses in $S$, which are the five that have the highest similarity scores ($r_{Alex,b}$) for Alex? What are their similarity scores? In case of ties between two businesses, choose the one with a smaller index. Do not write the index of the businesses, write their names using the file __business.csv__.
 

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the datasets
business_df = pd.read_csv('data/business.csv', header = None, names = ["Business"])
user_business_df = pd.read_csv('data/user-business.csv', header = None)

alex_idx, alex_erased_vals = 3, 100

user_business_df.iloc[alex_idx, :alex_erased_vals] = 0
alex_row = user_business_df.iloc[alex_idx, :].copy()

In [3]:
user_simil_matrix = cosine_similarity(user_business_df.iloc[:, alex_erased_vals:], 
                                      np.array(alex_row[alex_erased_vals:]).reshape(1, -1))
user_recomm_matrix = np.dot(user_simil_matrix.T, user_business_df).flatten()

display_top_k(business_df, user_recomm_matrix[:alex_erased_vals])

,Business,Similarity Score
1,Papi's Cuban & Caribbean Grill,43.039527
2,Seven Lamps,33.598188
3,Loca Luna,33.263225
4,Farm Burger,32.782940
5,Piece of Cake,12.626244


### Part B: item – item recommender system [10 points]
 
In an item-item recommender system, you need to find items that have similar ratings and recommend it to Alex. For all business b, compute
 
 $$r_{Alex,b} = \sum_{x ∈ business} cos-sim(x, b)·R_{Alex,x}$$ 
 
where $R$ is the ratings matrix and $cos-sim(x,b)$ is the cosine-similarity of each pair of businesses(excluding entries of Alex).  Here you are finding similar items and then multiplying it with Alex’s ratings for items. So the businesses that are similar to the businesses already visited by Alex will have the higher rating $r$
 
From all the businesses in $S$ (first 100 businesses), which are the five that have the highest similarity scores for Alex?  In case of ties between two businesses, choose the one with a smaller index. Again, hand in the names of the businesses and their similarity score.

In [4]:
item_simil_matrix = cosine_similarity(user_business_df.drop(labels = [alex_idx], axis = 0).T)
item_recomm_matrix = np.dot(alex_row, item_simil_matrix)

display_top_k(business_df, item_recomm_matrix[:alex_erased_vals])

,Business,Similarity Score
1,Papi's Cuban & Caribbean Grill,6.810937
2,Farm Burger,6.558815
3,Seven Lamps,6.440367
4,Loca Luna,5.852681
5,Piece of Cake,3.730178


### Part C: Latent hidden model recommender system [15 points]
 
Latent model recommender system is the most popular type of recommender system in the market today. Here we perform a matrix factorization of the ratings matrix $R$ into two matrices $U$ and $V$ where $U$ is considered as the user features matrix and $V$ is the movie features matrix. Note that the features are ‘hidden’ and need not be understandable to users. Hence the name latent hidden model. (refer slides for more information)
 
The latent model can be implemented by performing a singular value decomposition (SVD) that factors the matrix into three matrices
 
$$ R = U ΣV^T $$
 
where $R$ is user ratings matrix, $U$ is the user “features” matrix, $Σ$ is the diagonal matrix of singular values (essentially weights), and $V^T$ is the movie “features” matrix. $U$ and $V^T$ are orthogonal, and represent different things. $U$ represents how much users “like” each feature and VT represents how relevant each feature is to each business.

To get the lower rank approximation, we take these matrices and keep only the top $k$ features ($k$ factors), which we think of as the $k$ most important underlying taste and preference vectors.
 
With $k$ set to 10, perform SVD to identify the $U$ and $V$ matrices. You can then multiply the matrices to estimate the following
 
$$R^* = U Σ V^T$$

From the $R^*$ matrix, select the top 5 businesses for Alex in $S$ (first 100 businesses). In case of ties between two businesses, choose the one with a smaller index. Again, hand in the names of the businesses and their similarity score.
 
Hint: You can use SVD in scipy, numpy, or surprise package.

In [5]:
from scipy.linalg import svd

U, Sigma, V_t = svd(user_business_df)

f = 10
U_k, Sigma_k, V_t_k = U[:, :f], np.diag(Sigma[:f]), V_t[:f, :]
R_star = np.dot(U_k, np.dot(Sigma_k, V_t_k))

alex_row = R_star[alex_idx, :alex_erased_vals]

display_top_k(business_df, alex_row)

,Business,Similarity Score
1,Papi's Cuban & Caribbean Grill,1.190506
2,Loca Luna,0.876255
3,Farm Burger,0.857826
4,Seven Lamps,0.817947
5,Piece of Cake,0.299354


### Part D: bonus [10 points]

Your goal is to build a good recommendation system for Yelp with an ensemble of predictors. You can use any individual predictor and any method to combine them (it could be linear weighted combination or vote)

- Test set: 
    - 5 new users x the same 1000 businesses. Their records of the 100 first businesses are also erased.
    
- Submission: 
    - the prediction of the erased records which are 1s and 0s.
    - Submission format: sample_bonus_submission.csv, (5 rows, 100 columns, separator = comma, integers) Please make sure your raw text exactly matches the sample format. Otherwise you might have 0 points since we run auto grading.
    
- Evaluation metrics: 
    - Since the test set is sparse, i.e. most entries are 0s.  We use F1 score as our evaluation metric.
    - You can split a validation set out of the training set (for example, user 5-9 ) if you want to test your model. 
    
- Code and write-up:
    - Write your code for the test set in a separate jupyter notebook. At the top of the notebook, add brief write-ups to explain each predictor you used and how you combined them.
    - Your bonus points = max(10*min( yourF1 - 0.120.5 - 0.12, 1), 0)
        - This means that you will get some points as long as you attempt! For reference, a random guess(all as 1s) is 0.12. And 0.6 is pretty accurate.
      

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load the datasets
business_df = pd.read_csv('data/business.csv', header = None, names = ["Business"])
user_business_df = pd.read_csv('data/user-business.csv', header = None)
user_business_test_df = pd.read_csv('data/user-business_test.csv', header = None)

In [7]:
from sklearn.metrics import f1_score
import random

def normalize_arr(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

class my_recom_system:
    def __init__(self, erased_vals = 100, train_sample = 10, epochs = 10, 
                 weight_step_limit = 0.05, bias = 0.3):
        self.erased_vals = erased_vals
        self.weights = np.array([0.5, 0.5])
        self.bias = bias
        self.train_sample = train_sample
        self.epochs = epochs
        self.weight_step_limit = weight_step_limit

    def train(self):
        best_weights = self.weights.copy()
        prev_increm_idx = 0
        bias_increm_sign = 1
        best_f1_score = 0.0
        best_bias = self.bias
        
        random_user_train_sample = random.sample(range(len(user_business_df)), self.train_sample)
        for e in range(1, self.epochs + 1):
            curr_f1_score = 0.0
            
            for i in random_user_train_sample:
                user_row = user_business_df.iloc[i, :].copy()
                user_row_targets = user_row[:self.erased_vals].copy()
                user_row[:self.erased_vals] = 0

                predictions = self.predict(user_business_df, user_row, i)
                curr_f1_score += f1_score(user_row_targets, predictions, zero_division = 1) / \
                                 self.train_sample
            
            
            print("epoch: ", e, "\t F1 Score: ", curr_f1_score, 
                  "\t Bias-Weights: ", self.bias, " - ", self.weights)  
            
            if curr_f1_score < best_f1_score:
                self.weights = best_weights.copy()
                prev_increm_idx = (prev_increm_idx + 1) % 2
                self.bias = best_bias
                bias_increm_sign *= -1
            else :
                best_weights = self.weights.copy()
                best_f1_score = curr_f1_score
                best_bias = self.bias                
            
            weight_rand_increment = random.uniform(0, self.weight_step_limit)
            self.bias += bias_increm_sign * random.uniform(0, 0.05)
            self.weights -= weight_rand_increment
            self.weights[prev_increm_idx] += 2 * weight_rand_increment
    
        # Chose best weights and bias.
#         self.bias = best_bias
#         self.weights = best_weights
    
    def predict(self, user_df, user_row, i):
        # user system
        user_simil_matrix = cosine_similarity(user_df.iloc[:, self.erased_vals:], 
                                      np.array(user_row[self.erased_vals:]).reshape(1, -1))
        user_recomm_matrix = normalize_arr(np.dot(user_simil_matrix.T, user_df)
                                          ).flatten()[:self.erased_vals]
        
#         user_simil_matrix = cosine_similarity(user_df, np.array(user_row).reshape(1, -1))
#         user_recomm_matrix = normalize_arr(np.dot(user_simil_matrix.T, user_df)).flatten()[:self.erased_vals]

        # item system
    
        item_simil_matrix = cosine_similarity(user_df.drop(labels = [i], axis = 0).T)
        item_recomm_matrix = normalize_arr(np.dot(user_row, item_simil_matrix)
                                          )[:self.erased_vals]

#         item_simil_matrix = cosine_similarity(user_df.T)
#         item_recomm_matrix = normalize_arr(np.dot(user_row, item_simil_matrix))[:self.erased_vals]
        
        mixed_score = self.bias + user_recomm_matrix * self.weights[0] + \
                      item_recomm_matrix * self.weights[1]
        predictions = (mixed_score > 0.5).astype(int)
        
        return predictions
    
    
    def test(self, user_df):
        all_preds = np.zeros_like(user_df.iloc[:, :self.erased_vals])
        for i in range(len(user_df)):
            user_row = user_df.iloc[i, :].copy()
            predictions = self.predict(user_business_df, user_row, i)
            all_preds[i, :] = predictions
        
        pred_df = pd.DataFrame(all_preds)
        return pred_df

In [8]:
recom_sys = my_recom_system(train_sample = 50, epochs = 20)
recom_sys.train()

epoch:  1 	 F1 Score:  0.24683859999649474 	 Bias-Weights:  0.3  -  [0.5 0.5]
epoch:  2 	 F1 Score:  0.21737562437562438 	 Bias-Weights:  0.32749196447578327  -  [0.51987709 0.48012291]
epoch:  3 	 F1 Score:  0.28721428571428576 	 Bias-Weights:  0.25009684151025585  -  [0.47688532 0.52311468]
epoch:  4 	 F1 Score:  0.3407777777777778 	 Bias-Weights:  0.20539801250401565  -  [0.43550881 0.56449119]
epoch:  5 	 F1 Score:  0.3772727272727273 	 Bias-Weights:  0.17630011870866005  -  [0.4309508 0.5690492]
epoch:  6 	 F1 Score:  0.4387142857142858 	 Bias-Weights:  0.1582573046458343  -  [0.39150613 0.60849387]
epoch:  7 	 F1 Score:  0.437936507936508 	 Bias-Weights:  0.13494863237482072  -  [0.3914683 0.6085317]
epoch:  8 	 F1 Score:  0.3407777777777778 	 Bias-Weights:  0.20085027852706341  -  [0.44121234 0.55878766]
epoch:  9 	 F1 Score:  0.4314285714285715 	 Bias-Weights:  0.11444545071470597  -  [0.390782 0.609218]
epoch:  10 	 F1 Score:  0.35611111111111116 	 Bias-Weights:  0.17876587759

In [9]:
test_predictions = recom_sys.test(user_business_test_df)
test_predictions.to_csv("bonus_submission.csv", header = False, index = False)

display(test_predictions)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


### Turn in:
#### A2
    a) A Jupyter notebook a2.jpynb with the code and answers (if you work in a study group, write their names at the top to avoid any trouble in the plagiarism check. And we encourage you to write your code independently.)
    b) A a2.py exported from your .jpynb

#### A2-bonus
    c) bonus_submission.csv
    d) bonus.ipynb
    e) A bonus.py exported from your .jpynb (last time, some students submit an unexpected messy file with the raw content of Jupyter. Please make sure it is the exported one with codes only.)

__Please double check that you have all the required files submitted!__ Last time we received many regrading requests about this. In this and the following assignments, we will need to apply at least 20% penalty.